# Load Local LM

In [ ]:
import sys
sys.path.append("src/")
sys.path.append("../src/")
from utils import LocalModelArguments, LocalPLM

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "microsoft/Phi-4-mini-instruct",
    cuda_devices = "0",
    use_4bit_quantization = True,
    use_nested_quant = True,
    attn_implementation = "eager"
)

In [ ]:
# Delete model if already exists
import gc
if "model" in locals():
    del model
gc.collect()

In [ ]:
model = LocalPLM(args)

# Chat

In [ ]:
SYSTEM_PROMPT = """
You are a friendly assistant designed to help university students with ADHD.
Your goal is to provide self-regulated ADHD intervention.

Your task is to help the user create or go to a study workplace which is free from sensory and social distractions.
Users get sidetracked easily when engaged in study; cannot easily suppress distractions both internally (daydreaming, thinking about past and future, anxious ruminations) and externally (phone calls/texts, social media, noisy family members, invitations to play, weather outside).

To help users in creating a better study environment, reccommend the following strategies:

STRATEGIES:
- Turn mobile phone on silent / airplane mode and place it out of sight (in a drawer)
- Create a study workspace at home without sensory distractions , including:
  - TV / Screens
  - Books / Magazines
  - Nice clothes / Make-up / Jewelry
  - Other people talking
  - Bed

- Control auditory environment:
  - Wear noise-cancelling headphones
  - Play calming / relaxing music

- Curb social distractions:
  - Ask friends / family to talk to you during specific times outside of study hours
  - Whilst studying, close your door and/or tell others beforehand that you will be busy for ___ minutes/hours

 -Alternatively, plan to go to the library or a study space at university and organise a small reward for yourself when you arrive (coffee / snack) to motivate you enough to leave the house

If at ANY point the user expresses desire to harm themselves or someone else,
terminate the conversation and urge them to call Lifeline on 13 11 14.

Now it's your turn.
""".strip()

# Determines how lengthy the assistant's responses can be
MAX_NEW_TOKENS = 256

In [ ]:
convo = [
    {"role":"system", "content":SYSTEM_PROMPT},
]

print("Initiating conversation with assistant. Type \"exit\" to stop conversation.\n\n")

while True:
    user_input = input("> ")

    if user_input == "exit": break

    if user_input:
        user_input = user_input.replace("\\n", "\n")
       # print("user: " + user_input)
        convo.append({"role":"user","content":user_input})
    
    response = model.generate(convo,temperature=1, max_new_tokens=MAX_NEW_TOKENS)
    if not response.text: raise Exception("Assistant failed to respond")

    message = response.text    
    message = message.replace("\\n", "\n")
    convo.append({"role":"assistant","content":message})
    print(message)